<a href="https://colab.research.google.com/github/jeet1912/florence-2_ablationStudy/blob/main/code/mimic_cxr_ablation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gcsfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.6/199.6 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2025.3.0
    Uninstalling gcsfs-2025.3.0:
      Successfully uninstalled gcsfs-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.

In [ ]:
!pip install --upgrade google-cloud-storage

In [ ]:
!pip install iterative-stratification

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nv

In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [ ]:
import pandas as pd
import gcsfs
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from collections import defaultdict
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import io
import os
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import torch.optim as optim
import torch.nn as nn
import copy
import itertools
from evaluate import load
import gc
import time
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from google.colab import drive
from google.colab import auth
from google.cloud import storage

auth.authenticate_user()
drive.mount('/content/drive', force_remount=True)
plt.style.use('default')
sns.set_palette("colorblind")

Mounted at /content/drive


In [ ]:
class CXR(Dataset):
  def __init__(self, dataframe, processor, max_length):
    super().__init__()
    self.dataframe = dataframe.reset_index(drop=True)
    self.processor = processor
    self.max_length = max_length
    self.prompt = "List pathalogical findings for this chest X-ray:"
    self.storage_client = storage.Client(project='ablation-study')
  def __len__(self):
    return len(self.dataframe)

  def _loadImage(self,subject_id, study_id, dicom_id):
    try:
      bucket_name = "mimic-cxr-jpg-2.1.0.physionet.org"
      image_path = f"files/p{subject_id[:2]}/p{subject_id}/s{study_id}/{dicom_id}.jpg"
      bucket = self.storage_client.bucket(bucket_name, user_project='ablation-study')
      blob = bucket.blob(image_path)
      image_bytes = blob.download_as_bytes()
      image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
      return image
    except Exception as e:
      print(f"Error loading image {image_path}: {str(e)}")
      return None # Return None if image loading fails

  def __getitem__(self, index):
    row = self.dataframe.iloc[index]
    miniReport = str(row['mini_report'])
    subject = str(row['subject_id'])
    study = str(row['study_id'])
    dicom = str(row['dicom_id'])
    image = self._loadImage(subject_id=subject,study_id=study,dicom_id=dicom)
    inputs = self.processor(images=image, text=self.prompt, return_tensors="pt", padding="max_length",
                            truncation=True, max_length=self.max_length)
    labels = self.processor.tokenizer(miniReport, return_tensors="pt", padding="max_length",
                                      truncation=True, max_length=self.max_length)["input_ids"]

    return {
      "pixel_values": inputs["pixel_values"],  # Shape: [1, 3, H, W]
      "input_ids": inputs["input_ids"],        # Shape: [1, max_length]
      "attention_mask": inputs["attention_mask"],  # Shape: [1, max_length]
      "labels": labels                         # Shape: [1, max_length]
    }

In [ ]:
processor = AutoProcessor.from_pretrained("microsoft/git-base")

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/ds677_ablationStudy/train_split.csv')
test_df = pd.read_csv('/content/drive/MyDrive/ds677_ablationStudy/test_split.csv')
val_df = pd.read_csv('/content/drive/MyDrive/ds677_ablationStudy/val_split.csv')

max_length = max(len(processor.tokenizer.encode(report)) for report in train_df['mini_report'])
print(f"Max length of mini-reports: {max_length}")

train_dataset = CXR(train_df, processor, max_length)
val_dataset = CXR(val_df, processor, max_length)
test_dataset = CXR(test_df, processor, max_length)

Max length of mini-reports: 92


In [ ]:
print(train_dataset[0]['pixel_values'].shape)
print(train_dataset[0]['input_ids'].shape)
print(train_dataset[0]['attention_mask'].shape)
print(train_dataset[0]['labels'].shape)

torch.Size([1, 3, 224, 224])
torch.Size([1, 92])
torch.Size([1, 92])
torch.Size([1, 92])


In [ ]:
folder_path = '/content/drive/MyDrive/ds677_ablationStudy/results'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps")

In [ ]:
device

device(type='cuda')

In [ ]:
def collate_fn(batch):
    pixel_values = torch.cat([item["pixel_values"] for item in batch], dim=0)  # [batch_size, 3, H, W]
    input_ids = torch.cat([item["input_ids"] for item in batch], dim=0)        # [batch_size, max_length]
    attention_mask = torch.cat([item["attention_mask"] for item in batch], dim=0)  # [batch_size, max_length]
    labels = torch.cat([item["labels"] for item in batch], dim=0)               # [batch_size, max_length]

    #print(f"Batch shapes: pixel_values={pixel_values.shape}, input_ids={input_ids.shape}, "
    #      f"attention_mask={attention_mask.shape}, labels={labels.shape}")

    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")

In [ ]:
for name, module in model.named_modules():
  print(name)


git
git.embeddings
git.embeddings.word_embeddings
git.embeddings.position_embeddings
git.embeddings.LayerNorm
git.embeddings.dropout
git.image_encoder
git.image_encoder.vision_model
git.image_encoder.vision_model.embeddings
git.image_encoder.vision_model.embeddings.patch_embedding
git.image_encoder.vision_model.embeddings.position_embedding
git.image_encoder.vision_model.pre_layrnorm
git.image_encoder.vision_model.encoder
git.image_encoder.vision_model.encoder.layers
git.image_encoder.vision_model.encoder.layers.0
git.image_encoder.vision_model.encoder.layers.0.self_attn
git.image_encoder.vision_model.encoder.layers.0.self_attn.k_proj
git.image_encoder.vision_model.encoder.layers.0.self_attn.v_proj
git.image_encoder.vision_model.encoder.layers.0.self_attn.q_proj
git.image_encoder.vision_model.encoder.layers.0.self_attn.out_proj
git.image_encoder.vision_model.encoder.layers.0.layer_norm1
git.image_encoder.vision_model.encoder.layers.0.mlp
git.image_encoder.vision_model.encoder.layers.0

In [ ]:
# Define target modules for LoRA
vision_modules = ["q_proj", "k_proj", "v_proj", "out_proj", "fc1", "fc2"]
text_modules = ["query", "key", "value", "dense"]

# Ablation configurations
ablation_configs = [
    {"name": "lora_vision_only", "target_modules": vision_modules},
    {"name": "lora_text_only", "target_modules": text_modules},
]

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/ds677_ablationStudy/results/lora_target_modules",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    learning_rate=1e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=1,  # Save only the best model
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none",
    lr_scheduler_type="reduce_lr_on_plateau",
    optim="adamw_torch",
    label_names=["labels"]
)

bleu = load("bleu")

def evaluate_model(model, test_loader, processor, max_length, device):
    processor.tokenizer.padding_side = 'left'
    model.eval()
    test_loss = 0.0
    test_samples = 0
    predictions = []
    references = []

    with torch.no_grad():
        for batch in test_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            batch_size_actual = batch["pixel_values"].size(0)
            test_loss += loss.item() * batch_size_actual
            test_samples += batch_size_actual

            pixel_values = batch["pixel_values"]
            input_ids = batch["input_ids"]
            generated_ids = model.generate(
              pixel_values=pixel_values,
              input_ids=input_ids,
              max_new_tokens=50,  # Generate up to 50 new tokens
              do_sample=False,    # Use greedy decoding for reproducible results
              pad_token_id=processor.tokenizer.pad_token_id
            )
            generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
            predictions.extend([text.replace("List pathalogical findings for this chest X-ray:", "").strip() for text in generated_texts])

            label_ids = batch["labels"]
            reference_texts = processor.batch_decode(label_ids, skip_special_tokens=True)
            references.extend(reference_texts)

    avg_test_loss = test_loss / test_samples
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
    return avg_test_loss, bleu_score["bleu"]

In [ ]:
import gc
import torch
import time
import os
import pandas as pd
from contextlib import contextmanager

# A100 Resource Management Configuration
class A100ResourceManager:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.initial_memory = None

    def setup_memory_optimization(self):
        """Configure PyTorch for optimal A100 memory usage and speed"""
        if torch.cuda.is_available():
            # Enable memory fraction to prevent OOM
            torch.cuda.set_per_process_memory_fraction(0.90)  # Use 90% of GPU memory for speed

            # Enable memory mapping for large models
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True

            # Enable optimized attention for speed (when supported)
            try:
                torch.backends.cuda.enable_flash_sdp(True)
                torch.backends.cuda.enable_math_sdp(True)
                torch.backends.cuda.enable_mem_efficient_sdp(True)
            except AttributeError:
                # These may not be available in all PyTorch versions
                print("Advanced SDP backends not available, using default attention")

            # Optimize memory allocation for speed
            os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:256,expandable_segments:True,roundup_power2_divisions:16'

            # Performance optimizations
            os.environ['TORCH_CUDNN_V8_API_ENABLED'] = '1'
            os.environ['CUDA_LAUNCH_BLOCKING'] = '0'  # Async CUDA operations

            # Set CUDA memory growth strategy
            torch.cuda.empty_cache()

            # Record initial memory state
            self.initial_memory = torch.cuda.memory_allocated()

    def clear_memory(self, aggressive=False):
        """Comprehensive memory cleanup"""
        if torch.cuda.is_available():
            torch.cuda.synchronize()
            torch.cuda.empty_cache()

            if aggressive:
                # Force garbage collection multiple times
                for _ in range(3):
                    gc.collect()
                    torch.cuda.empty_cache()

        # CPU memory cleanup
        gc.collect()

    def get_memory_stats(self):
        """Get detailed memory statistics"""
        if torch.cuda.is_available():
            allocated = torch.cuda.memory_allocated() / 1024**2  # MB
            cached = torch.cuda.memory_reserved() / 1024**2  # MB
            max_allocated = torch.cuda.max_memory_allocated() / 1024**2  # MB

            return {
                "allocated_mb": allocated,
                "cached_mb": cached,
                "max_allocated_mb": max_allocated
            }
        return {"allocated_mb": 0, "cached_mb": 0, "max_allocated_mb": 0}

    @contextmanager
    def memory_managed_training(self):
        """Context manager for memory-safe training"""
        try:
            self.clear_memory(aggressive=True)
            yield
        finally:
            self.clear_memory(aggressive=True)

# Initialize resource manager
resource_manager = A100ResourceManager()
resource_manager.setup_memory_optimization()

# High-performance training arguments optimized for speed
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/GIT AblationStratergy/ablationStudy/results/lora_target_modules",
    per_device_train_batch_size=64,  # Increased for better GPU utilization
    per_device_eval_batch_size=64,   # Increased for faster evaluation
    gradient_accumulation_steps=1,   # Reduced for faster updates
    num_train_epochs=1,
    learning_rate=2e-4,  # Increased learning rate for faster convergence
    eval_strategy="no",  # Disabled for speed - only evaluate at end
    save_strategy="no",  # Disabled intermediate saves for speed
    logging_steps=50,    # Less frequent logging
    save_total_limit=1,
    load_best_model_at_end=False,  # Disabled for speed
    report_to="none",
    lr_scheduler_type="constant",  # Simpler scheduler for speed
    optim="adamw_torch_fused",  # Fused optimizer for A100
    label_names=["labels"],
    # Performance optimization flags
    dataloader_pin_memory=True,   # Enable for faster data transfer
    dataloader_num_workers=4,     # Parallel data loading
    remove_unused_columns=True,
    bf16=True,  # Use BF16 instead of FP16 for A100 (faster and more stable)
    tf32=True,  # Enable TF32 for even faster training on A100
    gradient_checkpointing=False,
    # Speed optimizations
    dataloader_drop_last=True,    # Avoid partial batches
    group_by_length=False,        # Disabled as it can slow down vision tasks
    prediction_loss_only=True,    # Only compute loss, skip other metrics
    disable_tqdm=False,           # Keep progress bar for monitoring
)

def configure_tokenizer_padding(processor):
    """Ensure consistent tokenizer padding configuration"""
    processor.tokenizer.padding_side = 'left'
    # Ensure pad token is properly set
    if processor.tokenizer.pad_token is None:
        processor.tokenizer.pad_token = processor.tokenizer.eos_token
    # Ensure pad_token_id is set
    if processor.tokenizer.pad_token_id is None:
        processor.tokenizer.pad_token_id = processor.tokenizer.eos_token_id

    print(f"Tokenizer configuration:")
    print(f"  - padding_side: {processor.tokenizer.padding_side}")
    print(f"  - pad_token: '{processor.tokenizer.pad_token}'")
    print(f"  - pad_token_id: {processor.tokenizer.pad_token_id}")

# Optimized evaluation function with memory management and fixed padding
def evaluate_model_optimized(model, test_loader, processor, max_length, device):
    """Memory-optimized evaluation function with proper padding handling"""
    # CRITICAL: Ensure left padding is set before any generation
    configure_tokenizer_padding(processor)

    model.eval()
    test_loss = 0.0
    test_samples = 0
    predictions = []
    references = []

    # Process in smaller chunks to avoid memory spikes
    chunk_size = 16

    with torch.no_grad():
        for i, batch in enumerate(test_loader):
            batch = {k: v.to(device) for k, v in batch.items()}

            # Split batch into smaller chunks if needed
            batch_size = batch["pixel_values"].size(0)

            for chunk_start in range(0, batch_size, chunk_size):
                chunk_end = min(chunk_start + chunk_size, batch_size)
                chunk_batch = {k: v[chunk_start:chunk_end] for k, v in batch.items()}

                outputs = model(**chunk_batch)
                loss = outputs.loss
                chunk_size_actual = chunk_batch["pixel_values"].size(0)
                test_loss += loss.item() * chunk_size_actual
                test_samples += chunk_size_actual

                # CRITICAL: Re-verify padding before generation (some operations can reset it)
                if processor.tokenizer.padding_side != 'left':
                    print(f"WARNING: Padding side was reset to {processor.tokenizer.padding_side}, fixing...")
                    configure_tokenizer_padding(processor)

                # Generate predictions in chunks
                pixel_values = chunk_batch["pixel_values"]
                input_ids = chunk_batch["input_ids"]

                # Ensure input_ids are properly formatted for left padding
                attention_mask = chunk_batch.get("attention_mask", None)

                generated_ids = model.generate(
                    pixel_values=pixel_values,
                    input_ids=input_ids,
                    attention_mask=attention_mask,  # Include attention mask
                    max_new_tokens=92,
                    do_sample=False,
                    pad_token_id=processor.tokenizer.pad_token_id,
                    eos_token_id=processor.tokenizer.eos_token_id,
                    use_cache=True  # Enable caching for better performance
                )

                generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
                predictions.extend([text.replace("List pathalogical findings for this chest X-ray:", "").strip() for text in generated_texts])

                label_ids = chunk_batch["labels"]
                reference_texts = processor.batch_decode(label_ids, skip_special_tokens=True)
                references.extend(reference_texts)

                # Clear intermediate tensors
                del chunk_batch, outputs, generated_ids

            # Clean up batch tensors
            del batch

            # Periodic memory cleanup during evaluation
            if i % 10 == 0:
                torch.cuda.empty_cache() if device.type == "cuda" else None

    avg_test_loss = test_loss / test_samples
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
    return avg_test_loss, bleu_score["bleu"]

# Fixed high-performance trainer class for speed optimization
class HighSpeedPaddingTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def training_step(self, model, inputs, num_items_in_batch=None):
        """
        Optimized training_step for maximum speed
        """
        # Ensure tokenizer padding is correct before each training step
        processing_class = getattr(self, 'processing_class', None)
        if processing_class is None:
            processing_class = getattr(self, 'tokenizer', None)

        if processing_class and hasattr(processing_class, 'padding_side'):
            if processing_class.padding_side != 'left':
                processing_class.padding_side = 'left'

        # Call parent's training_step with proper arguments
        if num_items_in_batch is not None:
            return super().training_step(model, inputs, num_items_in_batch)
        else:
            return super().training_step(model, inputs)

    def _save_checkpoint(self, model, trial, metrics=None):
        """Skip checkpointing for speed during training"""
        # Only save at the very end
        if self.state.epoch >= self.args.num_train_epochs - 0.01:
            super()._save_checkpoint(model, trial, metrics)

    def log(self, logs, start_time=None):
        """Reduced logging for speed with correct method signature"""
        # Only log every N steps to reduce I/O overhead
        if self.state.global_step % 50 == 0:
            # Call parent's log method with correct arguments
            if start_time is not None:
                super().log(logs, start_time)
            else:
                super().log(logs)

# Main ablation study loop with optimized resource management
results = []

for config_idx, config in enumerate(ablation_configs):
    print(f"\nRunning ablation experiment: {config['name']} ({config_idx + 1}/{len(ablation_configs)})")

    # Memory management context
    with resource_manager.memory_managed_training():
        # Pre-training memory stats
        pre_memory = resource_manager.get_memory_stats()
        print(f"Pre-training memory: {pre_memory['allocated_mb']:.1f}MB allocated")

        # Load model with speed optimization
        model = AutoModelForCausalLM.from_pretrained(
            "microsoft/git-base",
            torch_dtype=torch.bfloat16,  # Use bfloat16 for A100 speed
            device_map="auto" if torch.cuda.device_count() > 1 else None,
            low_cpu_mem_usage=True,
            use_cache=False  # Disable cache during training for memory efficiency
        )
        processor = AutoProcessor.from_pretrained("microsoft/git-base")

        # CRITICAL: Configure tokenizer padding immediately after loading
        configure_tokenizer_padding(processor)

        # Configure LoRA with speed considerations
        lora_config = LoraConfig(
            r=16,  # Increased rank for potentially better convergence with fewer epochs
            lora_alpha=32,  # Increased alpha proportionally
            target_modules=config["target_modules"],
            lora_dropout=0.05,  # Reduced dropout for speed
            bias="none",
            task_type="CAUSAL_LM",
            use_rslora=True,  # Use rank-stabilized LoRA for better training
            use_dora=False   # Disable DoRA for speed
        )

        peft_model = get_peft_model(model, lora_config)
        peft_model.to(device)

        # Ensure gradients are enabled for trainable parameters
        peft_model.train()
        for name, param in peft_model.named_parameters():
            if param.requires_grad:
                param.requires_grad_(True)

        peft_model.print_trainable_parameters()

        # Verify gradient requirements
        trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
        print(f"Verified trainable parameters: {trainable_params:,}")

        # Create high-performance data loaders
        train_loader = DataLoader(
            train_dataset,
            batch_size=64,  # Larger batch size for speed
            shuffle=True,
            collate_fn=collate_fn,
            pin_memory=True,  # Enable for faster GPU transfer
            num_workers=4,    # Parallel data loading
            persistent_workers=True,  # Keep workers alive between epochs
            drop_last=True,   # Avoid partial batches for consistent timing
            prefetch_factor=2  # Prefetch batches for speed
        )
        val_loader = DataLoader(
            val_dataset,
            batch_size=64,
            collate_fn=collate_fn,
            pin_memory=True,
            num_workers=4,
            persistent_workers=True,
            drop_last=True
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=32,  # Larger batch for faster evaluation
            collate_fn=collate_fn,
            pin_memory=True,
            num_workers=2,
            persistent_workers=True
        )

        # Memory checkpoint after model loading
        post_load_memory = resource_manager.get_memory_stats()
        print(f"Post-model-load memory: {post_load_memory['allocated_mb']:.1f}MB allocated")

        # Configure high-speed trainer
        trainer = HighSpeedPaddingTrainer(
            model=peft_model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            data_collator=collate_fn,
            processing_class=processor.tokenizer,
        )

        # Compile the model for speed (PyTorch 2.0+)
        try:
            peft_model = torch.compile(peft_model, mode="reduce-overhead")
            print("Model compiled successfully for speed optimization")
        except Exception as e:
            print(f"Model compilation failed: {e}. Continuing without compilation.")

        # Additional check: Ensure model is in training mode
        peft_model.train()

        # Verify that the model has trainable parameters before training
        trainable_count = sum(1 for p in peft_model.parameters() if p.requires_grad)
        if trainable_count == 0:
            raise RuntimeError(f"No trainable parameters found for {config['name']}")

        print(f"Ready to train with {trainable_count} parameter tensors requiring gradients")

        # Track training metrics
        start_time = time.time()
        memory_snapshots = []

        # Training with memory monitoring
        try:
            print("Starting training...")
            # CRITICAL: Ensure padding is set before training starts
            configure_tokenizer_padding(processor)

            train_result = trainer.train()

            # Collect training metrics
            train_losses = [log["loss"] for log in trainer.state.log_history if "loss" in log]
            val_losses = [log["eval_loss"] for log in trainer.state.log_history if "eval_loss" in log]

            # Memory usage tracking (sample during training)
            for epoch in range(min(1, len(train_losses))):  # Limit memory snapshots
                memory_stats = resource_manager.get_memory_stats()
                memory_snapshots.append(memory_stats['allocated_mb'])

            training_time = time.time() - start_time

            print("Training completed. Starting evaluation...")
            # CRITICAL: Reconfigure padding before evaluation
            configure_tokenizer_padding(processor)

            # Evaluation with memory management
            resource_manager.clear_memory()
            test_loss, bleu_score = evaluate_model_optimized(
                peft_model, test_loader, processor, max_length, device
            )

            # Store results
            results.append({
                "experiment": config["name"],
                "train_losses": train_losses,
                "val_losses": val_losses,
                "memory_usage_MB": memory_snapshots,
                "training_time_seconds": training_time,
                "test_loss": test_loss,
                "bleu_score": bleu_score,
                "trainable_parameters": peft_model.get_nb_trainable_parameters()[0],
                "max_memory_mb": post_load_memory['max_allocated_mb']
            })

            print(f"Experiment completed: BLEU={bleu_score:.4f}, Test Loss={test_loss:.4f}")

        except RuntimeError as e:
            print(f"Training failed with error: {e}")
            if "out of memory" in str(e).lower():
                print("Attempting recovery with smaller batch size...")
                # Could implement fallback strategy here
            results.append({
                "experiment": config["name"],
                "train_losses": [],
                "val_losses": [],
                "memory_usage_MB": [],
                "training_time_seconds": 0,
                "test_loss": float('inf'),
                "bleu_score": 0,
                "trainable_parameters": 0,
                "max_memory_mb": 0,
                "error": str(e)
            })

        finally:
            # Aggressive cleanup after each experiment
            print("Cleaning up resources...")
            del peft_model, model, trainer
            del train_loader, val_loader, test_loader
            resource_manager.clear_memory(aggressive=True)

            # Final memory check
            final_memory = resource_manager.get_memory_stats()
            print(f"Post-cleanup memory: {final_memory['allocated_mb']:.1f}MB allocated")

# Save results with additional memory metrics
folder_path = '/content/drive/MyDrive/GIT AblationStratergy/ablationStudy/results/LoRA'
os.makedirs(folder_path, exist_ok=True)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(folder_path, 'lora_target_modules_ablation_results_optimized.csv'), index=False)

print("\nAblation Study Results (Target Modules) - A100 Optimized:")
display_columns = ["experiment", "test_loss", "bleu_score", "training_time_seconds", "trainable_parameters", "max_memory_mb"]
print(results_df[display_columns])

# Memory usage summary
total_experiments = len([r for r in results if 'error' not in r])
print(f"\nResource Management Summary:")
print(f"- Completed experiments: {total_experiments}/{len(ablation_configs)}")
print(f"- Peak memory usage: {max([r.get('max_memory_mb', 0) for r in results]):.1f}MB")
print(f"- Average training time: {sum([r['training_time_seconds'] for r in results if r['training_time_seconds'] > 0]) / max(total_experiments, 1):.1f}s")


Running ablation experiment: lora_vision_only
trainable params: 1,327,104 || all params: 177,946,170 || trainable%: 0.7458


KeyboardInterrupt: 